In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
# Set the path to your main dataset directory
dataset_path = './archive (1)'

# Get a list of available command/folder names
commands = np.array(tf.io.gfile.listdir(str(dataset_path)))
print(commands)

We want to keep: down, go, left, right, stop, up
Others to consider: yes/no, numbers, on/off

Note: LICENSE, README.md, testing_list.txt, validation_list.txt are documents and _background_noise is additional data.

In [ ]:
commands = ["down", "go", "left", "right", "stop", "up", "yes", "no", "on", "off"] # commands to keep

# get file counts for each command
# Loop through each category and count the number of audio files
for command in commands:
    command_path = os.path.join(dataset_path, command)
    # Get the list of all .wav files in the command folder
    audio_files = [file for file in os.listdir(command_path)]
    print(f"Number of audio files in '{command}': {len(audio_files)}")

Number of audio files in 'down': 2359
Number of audio files in 'go': 2372
Number of audio files in 'left': 2353
Number of audio files in 'right': 2367
Number of audio files in 'stop': 2380
Number of audio files in 'up': 2375
Number of audio files in 'yes': 2377
Number of audio files in 'no': 2375
Number of audio files in 'on': 2367
Number of audio files in 'off': 2357


Each command has ~2375 examples.

From README:
The data was captured in a
variety of formats, for example Ogg Vorbis encoding for the web app, and then
converted to a 16-bit little-endian PCM-encoded WAVE file at a 16000 sample
rate. The audio was then trimmed to a one second length to align most
utterances, using the
extract_loudest_section
tool. The audio files were then screened for silence or incorrect words, and
arranged into folders by label.

Divided into directories this way, you can easily load the data using `keras.utils.audio_dataset_from_directory`. 

The audio clips are 1 second or less at 16kHz. The `output_sequence_length=16000` pads the short ones to exactly 1 second (and would trim longer ones) so that they can be easily batched.

In [10]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=dataset_path,
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)

Found 64727 files belonging to 31 classes.
Using 51782 files for training.
Using 12945 files for validation.

label names: ['_background_noise_' 'bed' 'bird' 'cat' 'dog' 'down' 'eight' 'five'
 'four' 'go' 'happy' 'house' 'left' 'marvin' 'nine' 'no' 'off' 'on' 'one'
 'right' 'seven' 'sheila' 'six' 'stop' 'three' 'tree' 'two' 'up' 'wow'
 'yes' 'zero']
